# Running Tune experiments with HEBOSearch

In this tutorial we introduce HEBO, while running a simple Ray Tune experiment. Tune’s Search Algorithms integrate with ZOOpt and, as a result, allow you to seamlessly scale up a HEBO optimization process - without sacrificing performance.

Heteroscadastic Evolutionary Bayesian Optimization (HEBO) does not rely on the gradient of the objective function, but instead, learns from samples of the search space. It is suitable for optimizing functions that are nondifferentiable, with many local minima, or even unknown but only testable. This necessarily makes the algorithm belong to the domain of "derivative-free optimization" and "black-box optimization".

In this example we minimize a simple objective to briefly demonstrate the usage of HEBO with Ray Tune via `HEBOSearch`. It's useful to keep in mind that despite the emphasis on machine learning experiments, Ray Tune optimizes any implicit or explicit objective. Here we assume `zoopt==0.4.1` library is installed. To learn more, please refer to the [HEBO website](https://github.com/huawei-noah/HEBO/tree/master/HEBO).

In [1]:
# !pip install ray[tune]
!pip install HEBO==0.3.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Using cached GPy-1.10.0-cp37-cp37m-macosx_10_9_x86_64.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 1.4 MB/s eta 0:00:00
  Using cached paramz-0.9.5-py3-none-any.whl
  Using cached cma-2.7.0-py2.py3-none-any.whl (239 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: cma
    Found existing installation: cma 3.2.2
    Uninstalling cma-3.2.2:
      Successfully uninstalled cma-3.2.2


Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [2]:
import time

import ray
from ray import tune
from ray.air import session
from ray.tune.search.hebo import HEBOSearch

Let's start by defining a simple evaluation function.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune two hyperparameters,
namely `width` and `height`, and `activation`.

In [3]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 1
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

Next, our ``objective`` function takes a Tune ``config``, evaluates the `score` of your experiment in a training loop,
and uses `session.report` to report the `score` back to Tune.

In [4]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        session.report({"iterations": step, "mean_loss": score})

In [5]:
ray.init(configure_logging=False)

Python version:,3.7.7
Ray version:,3.0.0.dev0
Dashboard:,http://127.0.0.1:8266


While defining the search algorithm, we may choose to provide an initial set of hyperparameters that we believe are especially promising or informative, and
pass this information as a helpful starting point for the `HyperOptSearch` object.

We also set the maximum concurrent trials to `8`.

In [6]:

previously_run_params = [
    {"width": 10, "height": 0, "activation": "relu"},
    {"width": 15, "height": -20, "activation": "tanh"},
]

known_rewards = [-189, -1144]

max_concurrent = 8

algo = HEBOSearch(
    metric="mean_loss",
    mode="min",
    points_to_evaluate=previously_run_params,
    evaluated_rewards=known_rewards,
    random_state_seed=123,
    max_concurrent=max_concurrent,
)

The number of samples is the number of hyperparameter combinations that will be tried out. This Tune run is set to `1000` samples.
(you can decrease this if it takes too long on your machine).

In [7]:
num_samples = 1000

In [8]:
# If 1000 samples take too long, you can reduce this number.
# We override this number here for our smoke tests.
num_samples = 10

Next we define a search space. The critical assumption is that the optimal hyperparamters live within this space. Yet, if the space is very large, then those hyperparameters may be difficult to find in a short amount of time.

In [9]:
search_config = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu", "tanh"])
}

Finally, we run the experiment to `"min"`imize the "mean_loss" of the `objective` by searching `search_config` via `algo`, `num_samples` times. This previous sentence is fully characterizes the search problem we aim to solve. With this in mind, notice how efficient it is to execute `tuner.fit()`.

In [10]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_config,
)
results = tuner.fit()

Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_67ec1a0a,TERMINATED,127.0.0.1:47498,relu,-100,0,10,100,11.4386,99,-10
objective_69ac3226,TERMINATED,127.0.0.1:47512,relu,0,10,10.1,100,10.9018,99,-10.1
objective_69ada8d6,TERMINATED,127.0.0.1:47513,relu,-50,15,5.06689,100,10.7768,99,-5.06689
objective_69af2530,TERMINATED,127.0.0.1:47514,tanh,50,5,6.19802,100,10.9312,99,-6.19802
objective_69b0a8a6,TERMINATED,127.0.0.1:47515,tanh,-25,7.5,-1.36711,100,10.7948,99,1.36711
objective_69b2375c,TERMINATED,127.0.0.1:47516,relu,75,17.5,17.5574,100,10.8966,99,-17.5574
objective_69b3bb9a,TERMINATED,127.0.0.1:47517,tanh,-75,12.5,-6.41984,100,10.9022,99,6.41984
objective_69b58f60,TERMINATED,127.0.0.1:47519,relu,25,2.5,12.8883,100,10.8995,99,-12.8883
objective_72267d26,TERMINATED,127.0.0.1:47563,tanh,-93.4243,16.2678,-8.28072,100,10.7101,99,8.28072
objective_75ed3e0e,TERMINATED,127.0.0.1:47568,tanh,28.8058,15.0428,3.94728,100,10.7472,99,-3.94728


Result for objective_67ec1a0a:
  date: 2022-07-22_15-34-37
  done: false
  experiment_id: b2cc3485f1024cbbbb5947a9acd341e9
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 10.0
  neg_mean_loss: -10.0
  node_ip: 127.0.0.1
  pid: 47498
  time_since_restore: 0.10423088073730469
  time_this_iter_s: 0.10423088073730469
  time_total_s: 0.10423088073730469
  timestamp: 1658500477
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 67ec1a0a
  warmup_time: 0.0028820037841796875
  
Result for objective_69ada8d6:
  date: 2022-07-22_15-34-40
  done: false
  experiment_id: 9ca732d0f466455cbaa1da6f553a17ab
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 15.0
  neg_mean_loss: -15.0
  node_ip: 127.0.0.1
  pid: 47513
  time_since_restore: 0.10410189628601074
  time_this_iter_s: 0.10410189628601074
  time_total_s: 0.10410189628601074
  timestamp: 1658500480
  timesteps_since_restore: 0
  training_iter

Here are the hyperparamters found to minimize the mean loss of the defined objective.

In [11]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'width': 16.267813332265522, 'height': -93.42430416543701, 'activation': 'tanh'}


In [12]:
ray.shutdown()